<목차>
- Segmetation을 위한 Foundation Model 모델 설계를 위한 Task 정의

### 이미지 Segmentation 모델(SAM)을 만들기 위한 Foundation Model을 설계하기 위해서 한 3가지 질문
1. What task will enable zero-shot generalization?  
2. What is the corresponding model architecture?
3. What data can power this task and model? 

- (처음 본 사물도 인식할 수 있는) zero-shot 일반화를 위한 Task는 어떤 Task인가?  
  --> `어떻하면 애매모호한 상황도 모두 대처할 수 있는 모델을 만들 수 있을까? Promptable Segmentation Task를 정의`.  
      점(point), 박스(box), 마스크(mask), 텍스트(text)를 입력으로 받을 수 있게 설계.  
  --> `모호한 Prompt가 주어졌을 때도 합리적인 mask를 출력해야 한다. 모호하다면 연관된 것을 다 확률로 나타낸다`.
  
- 이에 상응하는 모델 구조는 어떻게 되야 하는가?  
  --> Promt를 당연히 지원해야 한다.
  
- 어떤 데이터가 필요한가?  
  --> Promptable Segmentation Task를 위한 Segmentation Mask는 구하기 어렵다.

### 데이터 수집 과정
1. `Assisted-manual`
   - 공개된 Segmentation Dataset을 이용해 SAM 초기 학습
   - 웹 기반 인터페이스에서 초기 학습된 SAM을 이용해 데이터 생성
   - 새로 취득한 Data로 점진적 모델 학습(6회)
   - 120k 이미지로 부터 4.3M Mask 취득  
2. `Semi-automatic`
   - Mask의 종류를 다양화 하는 것을 목표로 함
   - 1단계에서 학습된 신뢰도 높은 Mask를 작업 화면에 표시
   - Annotator들은 그 외 Object를 작업
   - 새로 취득한 Data로 점진적 모델 학습(5회)
   - 180k 이미지로 부터 5.9M Mask 취득  
3. `Fully automatic`
   - 완전 자동화된 Annotation 단계
   - 이미지에 32 x 32 Regular Grid Point를 각 포인트 마다 대응되는 Mask가 할당된다.
   - 그 중에서 IoU가 높은 Mask만 남긴다.
   - 중복된 Mask 제거 등 후처리 작업 진행
   - 1.1M 이미지로 부터 1.1B Mask 취득

### 모델 구조

![image.png](https://github.com/facebookresearch/segment-anything/raw/main/assets/model_diagram.png?raw=true)
- Image encoder output인 image embedding, Prompt encoder output인 prompt embedding을 `두개의 embedding을 Lightweight Mask decoder에서 결합하여 Mask 예측`
- Prompt encoder + Mask decoder는 50ms 이내로 수행

- Powerful Image Encoder
- Prompt Encoder: Sparse/Dense Prompt로 나뉜다.
  - Sparse Prompt : Points, Boxes, Text를 임베딩 한다.
  - Dense Prompt : Mask을 임베딩 한다.
- Mask Decoder

※ 참고로 오픈소스에서는 Text를 임베딩하는 기능은 없다.

In [1]:
# D:\00_PILSA\segment-anything\segment_anything\modeling\prompt_encoder.py
class PromptEncoder(nn.Module):

    def __init__(
        self,
        embed_dim: int,
        image_embedding_size: Tuple[int, int],
        input_image_size: Tuple[int, int],
        mask_in_chans: int,
        activation: Type[nn.Module] = nn.GELU,
    ) -> None:
        super().__init__()
    
    def _get_batch_size(
        self,
        points: Optional[Tuple[torch.Tensor, torch.Tensor]],
        boxes: Optional[torch.Tensor],
        masks: Optional[torch.Tensor],
    ) -> int:
        """Gets the batch size of the output given the batch size of the input
        prompts."""
        if points is not None:
            return points[0].shape[0]
        elif boxes is not None:
            return boxes.shape[0]
        elif masks is not None:
            return masks.shape[0]
        else:
            return 1



    def _embed_masks(self, masks: torch.Tensor) -> torch.Tensor:
        """Embeds mask inputs."""
        mask_embedding = self.mask_downscaling(masks)
        return mask_embedding

    def _get_dive(self) -> torch.device:
        return self.point_embeddings[0].weight.device

    def forward(
        self,
        points: Optional[Tuple[torch.Tensor, torch.Tensor]],
        boxes: Optional[torch.Tensor],
        masks: Optional[torch.Tensor],
    ) -> Tuple[torch.Tensor, torch.Tensor]:
    """Embeds different types of prompts, returning both sparse and dense
        embeddings.

        Arguments:
          points (tuple(torch.Tensor, torch.Tensor) or none): point coordinates
            and labels to embed.
          boxes (torch.Tensor or none): boxes to embed
          masks (torch.Tensor or none): masks to embed

        Returns:
          torch.Tensor: sparse embeddings for the points and boxes, with shape
            BxNx(embed_dim), where N is determined by the number of input points
            and boxes.
            : Points, Boxes의 sparse 임베딩
          torch.Tensor: dense embeddings for the masks, in the shape
            Bx(embed_dim)x(embed_H)x(embed_W)
            : Mask 임베딩
    """
    bs = self._get_batch_size(points, boxes, masks)
    sparse_embeddings = torch.empty((bs, 0, self.embed_dim), device=self._get_device())

    if points is not None:
        coords, labels = points
        point_embeddings = self._embed_points(coords, labels, pad=(boxes is None))
        sparse_embeddings = torch.cat([sparse_embeddings, point_embeddings], dim=1)
    if boxes is not None:
        box_embeddings = self._embed_boxes(boxes)
        sparse_embeddings = torch.cat([sparse_embeddings, box_embeddings], dim=1)
    
    if masks is not None:
        dense_embeddings = self._embed_masks(masks)
    else:
        dense_embeddings = self.no_mask_embed.weight.reshape(1, -1, 1, 1)
            .expand(bs, -1, self.image_embedding_size[0], self.image_embedding_size[1])
    return sparse_embeddings, dense_embeddings


IndentationError: expected an indented block (267457045.py, line 11)

<예시>